In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [19]:
if 'gender' in data.columns:
    label_encoder = LabelEncoder()
    data['gender'] = label_encoder.fit_transform(data['gender'])


In [21]:
X = data.iloc[:, :-1].values  # All columns except the last
y = data.iloc[:, -1].values   # Last column


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [27]:
scaler = StandardScaler()


In [31]:
print("Initial dataset summary:\n", data.describe(include='all'))
print("Missing values in the dataset:\n", data.isnull().sum())


Initial dataset summary:
           Unnamed: 0        gender           age    height(cm)     height(m)  \
count   10725.000000  10725.000000  10698.000000  10686.000000  10725.000000   
unique           NaN           NaN           NaN           NaN           NaN   
top              NaN           NaN           NaN           NaN           NaN   
freq             NaN           NaN           NaN           NaN           NaN   
mean    11663.604289      0.480559     33.979062    165.625066      1.650228   
std      6813.962123      0.499645      8.024044      6.752179      0.120343   
min         1.000000      0.000000      0.000000    137.160000      0.000000   
25%      5959.000000      0.000000     29.000000    160.020000      1.600200   
50%     10695.000000      0.000000     32.000000    165.100000      1.651000   
75%     17453.000000      1.000000     37.000000    170.180000      1.701800   
max     23998.000000      1.000000    116.000000    190.500000      1.905000   

          wei

In [33]:
data.replace({'#DIV/0!': np.nan}, inplace=True)

In [35]:
data.dropna(inplace=True)


In [37]:
if 'gender' in data.columns:
    label_encoder = LabelEncoder()
    data['gender'] = label_encoder.fit_transform(data['gender'])

In [39]:
X = data.iloc[:, :-1].values  # All columns except the last
y = data.iloc[:, -1].values   # Last

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [43]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
def create_model():
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [47]:
def federated_averaging(models, weights):
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]
    total_weight = np.sum(weights)
    
    for model, weight in zip(models, weights):
        model_weights = model.get_weights()
        for i in range(len(avg_weights)):
            avg_weights[i] += model_weights[i] * weight
            
    for i in range(len(avg_weights)):
        avg_weights[i] /= total_weight

    return avg_weights

In [49]:
num_clients = 5
clients_data = np.array_split(X_train, num_clients)
clients_labels = np.array_split(y_train, num_clients)
client_weights = np.ones(num_clients)  # Assuming equal we

In [51]:
client_models = []
for client_data, client_labels in zip(clients_data, clients_labels):
    model = create_model()
    model.fit(client_data, client_labels, epochs=5, batch_size=32, verbose=0)
    client_models.append(model)


In [53]:
avg_weights = federated_averaging(client_models, client_weights)

In [55]:
global_model = create_model()
global_model.set_weights(avg_weights)

In [57]:
loss, accuracy = global_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

Test loss: -2126547.2500, Test accuracy: 0.0000


In [59]:
global_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 256)            │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,585 (139.00 KB)

 Trainable params: 35,585 (139.00 KB)

 Non-trainable params: 0 (0.00 B)